In [1]:
# coding: utf-8
import os
import xlrd
import re
import csv
import fasttext
import traceback
import random
import utils
import json
import jieba
import pandas as pd
from utils import fixText

def getConfig():
    config = {}
    with open('./config.json', 'r') as f:
        s = f.read()
        config = json.loads(s)
    return config

In [2]:
with open('data/train.tsv', 'r') as f:
    content = f.readlines()
    random.shuffle(content)
    train = content[0:len(content) // 5 * 4]
    test = content[len(content) // 5 * 4:]
    
    with open('data/old_train_set.txt', 'w') as out:
        for c in train:
            res = re.search(r'__label__(.*)[^\r\n]', c)
            label = c[res.span()[0]:res.span()[1]]
            c = re.sub(r'__label__(.*)', '', c)
            c = ''.join(re.split('\t| |\r|\n', c))
            c = fixText(c)
            out.write(c + ' ' + label + '\n')
    with open('data/old_test_set.txt', 'w') as out:
        for c in test:
            res = re.search(r'__label__(.*)[^\r\n]', c)
            label = c[res.span()[0]:res.span()[1]]
            c = re.sub(r'__label__(.*)', '', c)
            c = ''.join(re.split('\t| |\r|\n', c))
            c = fixText(c)
            # c = ''.join(c.split(' '))
            out.write(c + ' ' + label + '\n')
        

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.728 seconds.
Prefix dict has been built successfully.


In [3]:
config = getConfig()
model = fasttext.train_supervised(
    input = './data/old_train_set.txt',
    lr = config['lr'],
    dim = config['hidden_dim'],
    epoch = config['epoch']
)

Read 4M words
Number of words:  73112
Number of labels: 43
Progress: 100.0% words/sec/thread:  800887 lr:  0.000000 avg.loss:  0.437031 ETA:   0h 0m 0s


In [11]:
total = 0
true_tag = 0
with open('./data/old_test_set.txt', 'r') as f:
    content = f.readlines()
    total = len(content)
    for c in content:
        res = re.search(r'__label__(.*)[^\r\n]', c)
        label = c[res.span()[0]:res.span()[1]]
        c = re.sub(r'__label__(.*)', '', c)
        predict = model.predict(c[:-1])[0][0]
        
        print(model.predict(c[:-1]))
        # print(predict, label)
        if(predict == label):
            true_tag += 1
        break
#print('tag level 1 accurate: {}% ({}/{})'.format(true_tag * 100 / total, true_tag, total))

(('__label__街道',), array([0.90377563]))


In [24]:
import re
def resetLabelLv2(label):
    if(re.search(r'.+街(道)*', label)):
        label = re.search(r'.+街(道)*', label).group()
        if "街道" not in label:
            label = label.replace("街", "街道")
        if(re.search(r'.+[区|市]', label)):
            label = re.sub(re.search(r'.+[区|市]', label).group(), '', label)
    return label

def resetLabelLv3(label):
    if(re.search(r'[\u4e00-\u9fa50-9]+街(道)*', label)):
        label = re.search(r'街(道)*.*', label).group()
        label = re.sub(r'[(|（][\u4e00-\u9fa50-9]*[)|）]', '', label)
    return label


In [27]:
addr = '广州市白云区区鹤洞街办事处假装有这个分队(测试一下括号内）测试一下括号外（测试一下多个括号）'
addr2 = resetLabelLv2(addr)
addr3 = resetLabelLv3(addr)
print(addr2)
print(addr3)


鹤洞街道
街办事处假装有这个分队测试一下括号外


In [3]:
import json
from utils import resetLabelLv2, resetLabelLv3
with open('./data/train_set.json', 'r') as f:
    s = f.read()
    data_set = json.loads(s)
label1 = {}
label2 = {}
label3 = {}

for id in list(data_set.keys()):
    if data_set[id]['tag_level_1'] in label1:
        label1[data_set[id]['tag_level_1']] += 1
    else:
        label1[data_set[id]['tag_level_1']] = 1

    tag = resetLabelLv2(data_set[id]['tag_level_2'])
    if tag in label2:
        label2[tag] += 1
    else:
        label2[tag] = 1

    tag = resetLabelLv3(data_set[id]['tag_level_3'])
    if tag in label3:
        label3[tag] += 1
    else:
        label3[tag] = 1

In [19]:
print(len(label2))
print(label2)

459
{'荔湾区卫生健康局': 28772, '荔湾区住房建设和园林局': 7695, '荔湾区人力资源和社会保障局': 9840, '荔湾区市场监管局': 27365, '海龙街道': 2342, '广州地铁建设管理有限公司': 99, '大龙街道': 5, '白鹤洞街道': 4590, '区卫生健康局': 674, '东漖街道': 3440, '南源街道': 2361, 'undefine': 5819, '彩虹街道': 3053, '冲口街道': 2368, '花地街道': 1642, '西村街道': 3516, '东沙街道': 1555, '荔湾区公安分局': 2991, '石围塘街道': 4187, '荔湾区环保局': 188, '荔湾区消费者委员会': 1631, '茶滘街道': 2467, '荔湾区教育局': 4411, '华林街道': 2597, '金花街道': 3430, '昌华街道': 1169, '广州市第八人民医院': 50, '桥中街道': 2659, '棠下街道': 4, '逢源街道': 2666, '站前街道': 1889, '区人力资源社会保障局': 246, '嘉禾街道': 15, '岭南街道': 1237, '白云区卫生健康局': 330, '中南街道': 1241, '广州市疾控中心': 410, '六榕街道': 88, '北京街道': 4, '区文广旅体局': 500, '龙津街道': 1827, '荔湾区分局': 108, '花都区市场监督管理局': 16, '珠江管理': 67, '区水务局': 1328, '市局第三税务分局': 7, '交通警察支队': 405, '荔湾区民宗局': 47, '荔湾区科工信局': 468, '多宝街道': 1555, '同德街道': 38, '荔湾区民政局': 388, '沙面街道': 128, '荔湾区来穗局': 182, '广州医科大学附属第三医院': 43, '花都区交通运输局': 5, '区住建园林局': 51, '南石头街道': 18, '广州医科大学附属第一医院': 45, '荔湾区城管执法局': 521, '市建设工程安全监督站': 15, '黄埔区消委办': 29, '区人社局': 534, '员村街道': 5, '中山大学附属第一医院': 2, '荔湾区商务投促局':